<a href="https://colab.research.google.com/github/MariuszJarosz86/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [14]:
import pandas as pd
import numpy as np

import os, datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt

from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim ==1: y_train = to_categorical(y_train)
if y_test.ndim ==1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs',5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data',(X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(units = 1024, activation='relu'),  
    Dropout(0.3),

    Dense(units = 1024, activation='relu'),  
    Dropout(0.3),

    Dense(units = num_classes, activation='softmax'),                
  ])

In [9]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.7640 - accuracy: 0.2533 - val_loss: 1.1726 - val_accuracy: 0.6211
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.8847 - accuracy: 0.7217 - val_loss: 0.2311 - val_accuracy: 0.9376
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.3590 - accuracy: 0.8908 - val_loss: 0.0753 - val_accuracy: 0.9754
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2282 - accuracy: 0.9322 - val_loss: 0.0459 - val_accuracy: 0.9868
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1680 - accuracy: 0.9509 - val_loss: 0.0392 - val_accuracy: 0.9887


0.9630385487528345

In [11]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1265 - accuracy: 0.9630


0.9630385637283325

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=params['conv2D_cnn_block_one'], kernel_size=(3,3), activation='relu', input_shape = input_shape),
    Conv2D(filters=params['conv2D_cnn_block_one'], kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=params['conv2D_cnn_block_one'], kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=params['conv2D_cnn_block_two'], kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=params['conv2D_cnn_block_two'], kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=params['conv2D_cnn_block_two'], kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=params['conv2D_cnn_block_three'], kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=params['conv2D_cnn_block_three'], kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=params['conv2D_cnn_block_three'], kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(units = 1024, activation='relu'),  
    Dropout(params['dropout_dense_block_one']),

    Dense(units = 1024, activation='relu'),  
    Dropout(params['dropout_dense_block_two']),

    Dense(units = num_classes, activation='softmax'),                
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  
  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = int(params.get('epochs', 5)),
      verbose = 0
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, f'accurycy={accuracy}')

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [43]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 10),
    'epochs': hp.quniform('epochs', 5, 15, 3),

    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.1, 0.9),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.1, 0.9),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.1, 0.9),
    
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.1, 0.9),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.1, 0.9),

    'conv2D_cnn_block_one': hp.choice('conv2D_cnn_block_one', [32,64,128]),
    'conv2D_cnn_block_two': hp.choice('conv2D_cnn_block_two', [64,128,256]),
    'conv2D_cnn_block_three': hp.choice('conv2D_cnn_block_three', [128,256,512]),
    
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    100,
    Trials()
)

{'batch_size': 160.0, 'conv2D_cnn_block_one': 128, 'conv2D_cnn_block_three': 128, 'conv2D_cnn_block_two': 64, 'dropout_cnn_block_one': 0.40060825103334874, 'dropout_cnn_block_three': 0.1559553116241774, 'dropout_cnn_block_two': 0.3668264111484091, 'dropout_dense_block_one': 0.3296995946710388, 'dropout_dense_block_two': 0.14639867590012054, 'epochs': 9.0}
accurycy=0.9643990993499756
{'batch_size': 120.0, 'conv2D_cnn_block_one': 64, 'conv2D_cnn_block_three': 256, 'conv2D_cnn_block_two': 256, 'dropout_cnn_block_one': 0.6830752064103829, 'dropout_cnn_block_three': 0.8771650015371276, 'dropout_cnn_block_two': 0.13805153466940479, 'dropout_dense_block_one': 0.21124726505501892, 'dropout_dense_block_two': 0.117757363345038, 'epochs': 12.0}
accurycy=0.9335600733757019
{'batch_size': 100.0, 'conv2D_cnn_block_one': 128, 'conv2D_cnn_block_three': 128, 'conv2D_cnn_block_two': 64, 'dropout_cnn_block_one': 0.6338065502178336, 'dropout_cnn_block_three': 0.4067142900401022, 'dropout_cnn_block_two': 0